# カテゴリカル分布
goodかbadの二択問題で、goodの確率が $\mu$ であるとき、goodである確率とbadである確率をまとめて書くと
<span id="eq1">
\begin{equation}
P(t)=\mu^t(1-\mu)^{1-t}
\end{equation}
</span>
と書けた。

次に、三択以上の確率分布を考えよう。サイコロの目のように、出目が6通りあるとする。このときの出目を、以下のようなone-hot表現で表すことにする。
$$\boldsymbol{x}=(0, 0, 1, 0, 0, 0)^\top$$

このとき、one-hot表現の出目ベクトル $\boldsymbol{x}$ の確率は
<span id="eq2">
\begin{equation}
P(\boldsymbol{x}|\boldsymbol{\mu})=\prod_{m=1}^6 \mu_m^{x_m}
\end{equation}
</span>
となる。 $\boldsymbol{\mu}=(\mu_1,\mu_2,\cdots)$ はパラメータベクトルである。

$K$ 回サイコロを振ったデータをまとめて $\boldsymbol{\mathsf{X}}$ と書くことにすると、このモデルから $\boldsymbol{\mathsf{X}}$ が発生する同時確率は
<span id="eq3">
\begin{equation}
P(\boldsymbol{\mathsf{X}}|\boldsymbol{\mu})=\prod_{k=1}^K\prod_{m=1}^6 \mu_m^{x_{km}}
\end{equation}
</span>

この同時確率を $\boldsymbol{\mu}$ の関数とみなし、これを尤度関数という。よって対数尤度関数は
<span id="eq4">
\begin{equation}
\ln L(\boldsymbol{\mu})=\ln P(\boldsymbol{\mathsf{X}}|\boldsymbol{\mu})=\sum_{k=1}^K\sum_{m=1}^6 x_{km}\ln\mu_m
\end{equation}
</span>
この関数を最大にするような $\boldsymbol{\mu}$ を求めればよい。

ところで、$m$ の目が $c_m$ 回出たとすると、 $c_m=\sum_{k=1}^K x_{km}$ だから
<span id="eq5">
\begin{equation}
\ln L(\boldsymbol{\mu})=\sum_{m=1}^6 c_m\ln\mu_m
\end{equation}
</span>
と書いてもよい。

## 勾配法によるパラメータ推定
尤度を最大にする $\boldsymbol{\mu}$ は解析的に求まり、
<span id="eq-ml">
\begin{equation}
\hat{\mu}_m=\frac{c_m}{K}
\end{equation}
</span>
である([課題](#kadai2))。わざわざパラメータ推定を勾配法で行う必要は全くないのだが、今回は練習のために敢えて勾配法で求めてみる。

6通りの出目を持つサイコロを100回振った結果から、パラメータ(=出目の確率)を推定しよう。まず、$\boldsymbol{\mu}$ の真値を決めておく。全部等確率だとおもしろくないので、2の目の確率を0.5, そのほかを0.1とする。

In [1]:
import numpy as np

mu_true = [0.1, 0.5, 0.1, 0.1, 0.1, 0.1]
c = np.random.multinomial(100, mu_true)
c

array([10, 59, 10,  5,  8,  8])

次に、[式(2)](#eq3)の確率分布にしたがって100回サイコロを振る。

In [2]:
xk = np.random.choice(6,100,p=mu_true)
xk+1

array([4, 2, 1, 2, 2, 2, 2, 2, 2, 6, 3, 3, 5, 2, 4, 2, 3, 2, 5, 5, 2, 2,
       5, 2, 2, 5, 4, 6, 6, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 3, 1, 3, 4, 1,
       4, 5, 3, 1, 2, 6, 2, 6, 3, 1, 4, 4, 2, 2, 2, 2, 2, 5, 2, 6, 6, 4,
       2, 2, 2, 2, 2, 1, 2, 2, 6, 2, 6, 3, 1, 1, 4, 3, 3, 2, 2, 2, 2, 3,
       2, 3, 3, 2, 6, 1, 5, 3, 2, 4, 5, 1])

出目の回数は bincount 関数で数えられる。

In [3]:
c = np.bincount(xk)
c

array([12, 45, 14, 10,  9, 10])

xkをone-hot表現に変換しよう。

In [4]:
X = np.identity(6)[xk]
X[0:10,:]

array([[0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.]])

[式(4)](#eq4)にしたがい、Xがデータとして与えられたときの対数尤度関数は次のように定義できる。

In [5]:
def categorical_likelihood(mu):
    return np.sum(X * np.log(mu))

one-hotベクトルでなく、出目の回数cがデータとして与えられた場合の対数尤度関数は、[式(5)](#eq5)より次のように定義できる。

In [6]:
def multinomial_likelihood(mu):
    return np.sum(c * np.log(mu))

これらが同じ結果を与えることを確認しておこう。適当な $\mu$ に対して

In [7]:
mu = np.array([0.2,0.1,0.2,0.1,0.3,0.1])
print(categorical_likelihood(mu))
print(multinomial_likelihood(mu))

-202.349172006833
-202.349172006833


同じ結果になった。以下ではデータとしてcを与えることにする。

In [8]:
# 「ゼロから作るDeep Learning」 p.104

def numerical_gradient(f, x):
    h = 1.0e-4
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        x[idx] = tmp_val - h
        fxh2 = f(x)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
        
    return grad

In [9]:
# 「ゼロから作るDeep Learning」 p.107

def gradient_descent(f, init_x, lr=0.001, step_num=100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
    return x

損失関数は負の対数尤度で定義し、categorical_entropy_lossという名前を付けておく。

ここで1つコツがある。 $\mu_1,\mu_2,\cdots$ の総和は1でなければならないが、モデルパラメータにそのような制約を入れるのは少し厄介である。そこで、パラメータは和が1にならなくてもよいこととし、それらを $\mu^\text{raw}_{1}, \mu^\text{raw}_{2},\cdots$ と書く(つまり $\mu^\text{raw}_{1}$ 等は確率ではない)。そして、確率は
$$\mu_m = \frac{\mu^\text{raw}_{m}}{\sum_{m=1}^6\mu^\text{raw}_{m}}$$
のようにして計算する。

In [10]:
def categorical_entropy_loss(mu_raw):
    mu = mu_raw / np.sum(mu_raw)
    return -multinomial_likelihood(mu)

では、パラメータを推定してみよう。

In [11]:
mu_raw = gradient_descent(categorical_entropy_loss, np.random.rand(6),lr=0.01)
mu = mu_raw / np.sum(mu_raw)
mu

array([0.12000001, 0.44999995, 0.14000002, 0.10000001, 0.09      ,
       0.10000001])

# 多クラスロジスティック回帰
ロジスティック回帰を多クラス問題に拡張しよう。多クラスロジスティック回帰モデルは、線形予測子 $\boldsymbol{z}=(z_1 z_2 \cdots z_M)^\top$ および softmax 関数 $\text{softmax}(z_m)=\frac{\exp(z_m)}{\sum_j \exp(z_j)}$ により以下のように書くことができる ($m$ はクラス番号)。
<span id="multiclasslogistic">
\begin{align*}
z_m&=\boldsymbol{w_m}^\top\boldsymbol{x} \\
y_m&=\text{softmax}(z_m)=\frac{\exp(z_m)}{\sum_j \exp(z_j)}
\end{align*}
</span>

ロジスティック回帰で仮定した確率の[式(1)](#eq1)を[式(2)](#eq2)に置き換える。説明変数 $\boldsymbol{x}$ によって多クラスロジスティック回帰モデルで予測されるクラス ${\cal C}$ の確率は
\begin{equation}
P(\boldsymbol{t}|\boldsymbol{w})=\prod_m y_m^{t_m}
\end{equation}
ただし $\boldsymbol{t}=(t_1 t_2 \cdots)^\top$ はクラス ${\cal C}$ の one-hot 表現である。

データセット $\boldsymbol{\mathsf{X}}=(\boldsymbol{x}_1 \boldsymbol{x}_2 \cdots)$,
$\boldsymbol{\mathsf{T}}=(\boldsymbol{t}_1 \boldsymbol{t}_2 \cdots)$ に対し、 $\boldsymbol{x}_1, \boldsymbol{x}_2 \cdots$ のそれぞれが $\boldsymbol{t}_1, \boldsymbol{t}_2 \cdots$ になる同時確率は、それぞれの確率の積になるから
\begin{equation}
P(\boldsymbol{\mathsf{T}}|\boldsymbol{w})=\prod_k \prod_m y_{km}^{t_{km}}
\end{equation}

よって、交差エントロピー(=負の対数尤度)は
<span id="eq-ent">
\begin{equation}
E=-\ln P(\boldsymbol{\mathsf{T}}|\boldsymbol{w})=-\sum_k \sum_m t_{km} \ln y_{km}
\end{equation}
</span>
となる。


## iris (アヤメの判別)
ロジスティック回帰を用いてアヤメの花の寸法から3種類のアヤメを判別する問題を解いてみよう。irisデータセットについては各自調べておいてください。

In [12]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
T = np.identity(3)[iris.target]

説明変数は4つ。

In [13]:
X[0:10,:]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

目的変数tは3種類のアヤメの種類のone-hot表現。

In [14]:
T[0:5,:] , T[145:,:]

(array([[1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]]), array([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]]))

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.5, random_state=0)

In [16]:
X_train = X[0::2] # 偶数番目
X_test = X[1::2] # 奇数番目
T_train = T[0::2]
T_test = T[1::2]

softmax関数を定義する。戻り値はクラス数を $M$ とすると $M$ 次元のベクトルになる。

In [17]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

[式(9)](#eq-ent)に従って損失関数を定義する。

In [18]:
def categorical_cross_entropy_loss(W):

    
    

パラメータ $\boldsymbol{W}=(\boldsymbol{w}_1 \boldsymbol{w}_2 \boldsymbol{w}_3)$ は今度は行列になるので、多次元配列に対応したバージョンの numerical_gradient を使う。([ここ](https://github.com/oreilly-japan/deep-learning-from-scratch)から入手できます)

In [19]:
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 値を元に戻す
        it.iternext()   
        
    return grad

In [20]:
X_train_a = np.column_stack((np.ones(X_train.shape[0]),X_train))
X_test_a = np.column_stack((np.ones(X_test.shape[0]),X_test))

In [21]:
W=gradient_descent(categorical_cross_entropy_loss, np.random.rand(5,3), lr=0.001,step_num=2000)

テストデータに対してロジスティック回帰モデルによる予測を行う。各データが各クラスの花である確率は次のように計算できる。

In [22]:
Y=[softmax(x) for x in X_test_a.dot(W)]
Y

[array([9.74597707e-01, 2.54022892e-02, 4.22062668e-09]),
 array([9.74765757e-01, 2.52342320e-02, 1.06202665e-08]),
 array([9.93588421e-01, 6.41157920e-03, 2.90446617e-10]),
 array([9.88059504e-01, 1.19404949e-02, 9.16534435e-10]),
 array([9.76139723e-01, 2.38602737e-02, 2.84919748e-09]),
 array([9.84665827e-01, 1.53341701e-02, 2.81968790e-09]),
 array([9.89087634e-01, 1.09123640e-02, 2.05323312e-09]),
 array([9.98932441e-01, 1.06755902e-03, 3.82799086e-12]),
 array([9.92340290e-01, 7.65970996e-03, 4.10561595e-10]),
 array([9.95403502e-01, 4.59649776e-03, 1.83612685e-10]),
 array([9.93660028e-01, 6.33997175e-03, 4.94902921e-10]),
 array([9.68932838e-01, 3.10671445e-02, 1.76149657e-08]),
 array([9.57959063e-01, 4.20409261e-02, 1.05491990e-08]),
 array([9.90695401e-01, 9.30459851e-03, 3.38984426e-10]),
 array([9.74660588e-01, 2.53394029e-02, 9.28441528e-09]),
 array([9.86256140e-01, 1.37438594e-02, 9.07529165e-10]),
 array([9.98797471e-01, 1.20252892e-03, 3.19223693e-12]),
 array([9.9078

$\boldsymbol{y}=(y_1 y_2 y_3)$ の中で確率が最大のクラスが認識結果となる。

In [23]:
result = np.array([np.argmax(y) for y in Y])
result

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2])

正解精度を求める。

In [24]:
np.sum(np.equal(result, [np.argmax(t) for t in T_test])) / T_test.shape[0]

0.9466666666666667

In [25]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='lbfgs',multi_class='multinomial')
logreg.fit(iris.data,iris.target)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [26]:
-np.sum(T_train * logreg.predict_log_proba(X_train))

8.865244646419518

In [27]:
-np.sum(T_train * np.log(Y))

9.83270063197948

# 課題
1. (基本) [softmax関数の式](#multiclasslogistic)で、クラスの数を2とするとロジスティック関数と同じになることを示せ。
1. <span id="kadai2">(発展)</span> [カテゴリカル分布の最尤推定式](#eq-ml)を証明せよ。(ヒント: この問題は制約付き最大化問題になるのでLagrangeの未定乗数法を使うと解ける。)
1. (基本) 上で未定義のまま残しておいた categorical_cross_entropy_loss 関数を完成させ、アヤメの判別を実行せよ。